# {{ALGORITHM_NAME}} with OpenEO

This notebook demonstrates {{ALGORITHM_DESCRIPTION}} using Sentinel-2 imagery with the OpenEO API.

## Overview

In this notebook, we will:
1. Load parameter sets using the simple OpenEO UDP parameter management system
2. Connect to an OpenEO backend service 
3. Define parameters from predefined sets
4. Load Sentinel-2 L2A imagery with validated parameters
5. Implement the {{ALGORITHM_NAME}} algorithm
6. Create visualizations and export results

### Applications

{{ALGORITHM_APPLICATIONS}}

## Import Required Libraries

Import the necessary Python libraries for data processing, visualization, and parameter management.

In [ ]:
# Standard OpenEO and visualization imports
import openeo
import matplotlib.pyplot as plt
from PIL import Image
from openeo.processes import array_create, if_, and_
from openeo.api.process import Parameter

# OpenEO UDP simple parameter management system
from openeo_udp import ParameterManager
from openeo_udp.config import get_connection, load_endpoint_config

## Load Parameters and Configuration

Load algorithm-specific parameter sets from the co-located .py file and set up endpoint configuration.

In [ ]:
# Initialize parameter manager for this notebook
import os
current_notebook = os.path.basename(__file__) if '__file__' in globals() else 'current_notebook.ipynb'
param_manager = ParameterManager(current_notebook)

# Show available parameter sets
available_sets = param_manager.list_parameter_sets()
print(f"Available parameter sets: {available_sets}")

# Use the first parameter set (or specify one)
if available_sets:
    param_manager.use_parameter_set(available_sets[0])
    current_params = param_manager.get_parameter_set()
    print(f"Using parameter set: {available_sets[0]}")
    print(f"Parameters: {current_params}")
else:
    print("No parameter sets found")

# Load endpoint configuration
endpoint_config = load_endpoint_config()
available_endpoints = [name for name, config in endpoint_config['endpoints'].items() 
                      if config.get('enabled', True)]
print(f"Available endpoints: {available_endpoints}")

## Select Endpoint and Connect

Select an endpoint and establish connection with proper authentication.

In [ ]:
# Select endpoint
selected_endpoint = available_endpoints[0] if available_endpoints else "eopf_explorer"
print(f"Selected endpoint: {selected_endpoint}")

# Establish connection
try:
    connection = get_connection(selected_endpoint)
    print(f"Successfully connected to {selected_endpoint}")
except Exception as e:
    print(f"Connection failed: {e}")
    print("Falling back to first available endpoint...")
    connection = get_connection()
    
# Set the endpoint for parameter formatting
param_manager.set_endpoint(selected_endpoint)

## Load Algorithm Parameters

Load and validate algorithm-specific parameters with endpoint-compatible formatting.

In [ ]:
# Load standard parameters (automatically formatted for selected endpoint)
bounding_box = param_manager.get_parameter('bounding_box')
time = param_manager.get_parameter('time')
bands = param_manager.get_parameter('bands')
collection = param_manager.get_parameter('collection')

# Display parameter values
print("Parameter Configuration:")
print(f"Spatial extent: {bounding_box.default}")
print(f"Temporal range: {time.default}")
print(f"Bands: {bands.default}")
print(f"Collection: {collection.default}")

# Load optional parameters if available
try:
    cloud_cover = param_manager.get_parameter('cloud_cover')
    print(f"Cloud cover threshold: {cloud_cover.default}%")
except ValueError:
    # Use default if parameter not found
    cloud_cover = Parameter("cloud_cover", default=30, optional=True)
    print("Using default cloud cover threshold: 30%")

# Show other parameter sets available
if len(available_sets) > 1:
    print(f"\nOther parameter sets available: {available_sets[1:]}")
    print("To use a different set, call: param_manager.use_parameter_set('set_name')")

## Load Sentinel-2 Data

Load Sentinel-2 L2A (atmospherically corrected) data with validated parameters and endpoint-specific formatting.

In [ ]:
print(f"Loading collection: {collection.default}")
print(f"Using bands: {bands.default}")

# Load data with cloud filtering
s2cube = connection.load_collection(
    collection.default,
    temporal_extent=time.default,
    spatial_extent=bounding_box.default,
    bands=bands.default,
    properties={
        "eo:cloud_cover": lambda x: x < cloud_cover.default,
    },
)

print("Sentinel-2 data loaded successfully")

## Implement {{ALGORITHM_NAME}} Algorithm

{{ALGORITHM_IMPLEMENTATION_DESCRIPTION}}

*Note: Replace this section with your specific algorithm implementation.*

In [ ]:
def algorithm_function(data):
    """
    Implement your algorithm here.
    
    Args:
        data: Input data array from Sentinel-2 bands
        
    Returns:
        Processed result for visualization
    """
    # TODO: Replace with actual algorithm implementation
    # This is a placeholder that extracts the first three bands for RGB visualization
    
    # Extract bands (adapt indices based on your algorithm's needs)
    b1, b2, b3 = data[0], data[1], data[2]
    
    # Example processing: create true color composite
    true_color = array_create([b3 * 3, b2 * 3, b1 * 3])  # RGB composite
    
    return true_color

# Apply algorithm to the data
result_image = s2cube.apply_dimension(dimension="bands", process=algorithm_function)

print("Algorithm applied successfully")

## Visualization and Export

Process the results for visualization and create output products.

In [ ]:
# Scale for visualization
result_scaled = result_image.linear_scale_range(
    input_min=0,
    input_max=0.8, 
    output_min=0,
    output_max=255
)

# Save as PNG
result_png = result_scaled.save_result("PNG")

print("Results prepared for visualization")

## Download and Display Results

Download a sample of the results for local visualization.

In [ ]:
# Define output filename based on current parameter set
current_set_name = param_manager._current_set or "result"
output_filename = f"{current_set_name}_result.png"

# Download result
try:
    connection.download(
        {
            "process_graph": result_png.flat_graph(),
            "parameters": [
                time.to_dict(),
                bounding_box.to_dict(),
                bands.to_dict(),
            ]
        },
        output_filename,
    )
    
    # Display the result
    result_img = Image.open(output_filename)
    plt.figure(figsize=(12, 10))
    plt.imshow(result_img)
    plt.title(f"Algorithm Results - {current_set_name}", fontsize=16)
    plt.axis("off")
    plt.show()
    
    print(f"Results saved to: {output_filename}")
    
except Exception as e:
    print(f"Download failed: {e}")

## Test with Alternative Parameter Sets

Test the algorithm with different parameter sets defined in the parameter file.

In [ ]:
# Show all available parameter sets
print("Available parameter sets:")
for set_name in available_sets:
    params = param_manager.get_parameter_set(set_name)
    bbox = params.get('bounding_box', {})
    time_range = params.get('time', [])
    print(f"\n{set_name}:")
    print(f"  Location: {bbox}")
    print(f"  Time: {time_range}")

print("\n" + "=" * 40)
print("To test with a different parameter set:")
print("1. Call: param_manager.use_parameter_set('set_name')")
print("2. Re-run the parameter loading cell")
print("3. Re-run the data loading and processing cells")

## Export Process Graph

Export the algorithm as a reusable OpenEO User-Defined Process.

In [ ]:
# Create process graph for reuse
import json

# Use current parameter set name for process ID
process_id = param_manager._current_set or "algorithm"

process_graph = {
    "process_graph": result_png.flat_graph(),
    "parameters": [
        time.to_dict(),
        bounding_box.to_dict(),
        bands.to_dict(),
    ],
    "id": process_id,
    "summary": f"Algorithm with {process_id} parameters",
    "description": "OpenEO UDP algorithm",
    "experimental": True,
}

# Save process graph
process_graph_filename = f"{process_id}_process_graph.json"
with open(process_graph_filename, "w") as f:
    json.dump(process_graph, f, indent=2)

print(f"Process graph exported to: {process_graph_filename}")
print(f"Process ID: {process_graph['id']}")

## Summary

This notebook demonstrated the algorithm using the simplified OpenEO UDP parameter management system.

### Key Features:
- ✅ Simple parameter sets in co-located .py files
- ✅ Multiple parameter sets per algorithm  
- ✅ Automatic endpoint connection management
- ✅ Parameter formatting for different OpenEO backends
- ✅ Easy switching between parameter sets
- ✅ Process graph export with embedded parameters

### Parameter File Structure:
```python
def get_parameters():
    return {
        "set_name": {
            "bounding_box": {...},
            "time": [...],
            "bands": [...],
            "collection": "...",
            "cloud_cover": 30
        }
    }
```

### Usage:
1. **Load parameters**: `ParameterManager('notebook.ipynb')`
2. **List sets**: `param_manager.list_parameter_sets()`
3. **Use set**: `param_manager.use_parameter_set('set_name')`
4. **Get parameter**: `param_manager.get_parameter('bounding_box')`

For more information, see the OpenEO UDP documentation.